**This notebook is used to interact with Mongodb for various**

In [2]:
import os
import json

import pandas as pd
from dotenv import load_dotenv
from pymongo import MongoClient
from bunnet import init_bunnet
from stock_analysis.schema.db import *

load_dotenv()

True

In [3]:
# creating async client
client = MongoClient(os.environ["MONGODB_CONNECTION_STRING"])

# Initialize beanie
# NOTE - db name is stock-repo-db & document_models will eventually become collection
init_bunnet(database=client["stock-repo-db"], document_models=[NiftyIndex,NiftySector])

In [15]:
# populating nifty index data into nifty-index collection
data = pd.read_csv("data/nifty_index_data.csv")

index_data_collection = [
    NiftyIndex(
        symbol=row["symbol"],
        Nifty50=row["Nifty50"],
        Nifty100=row["Nifty100"],
        Nifty200=row["Nifty200"],
        Nifty500=row["Nifty500"],
        NiftySmallcap50=row["NiftySmallcap50"],
        NiftySmallcap250=row["NiftySmallcap250"],
        NiftyMidcap50=row["NiftyMidcap50"],
        NiftyMidcap150=row["NiftyMidcap150"],
        NiftyMidcap400=row["NiftyMidcap400"],
        NiftyNext50=row["NiftyNext50"],
    )
    for _, row in data.iterrows()
]

NiftyIndex.insert_many(index_data_collection)

In [16]:
# populating nifty sector data into nifty-sector collection
with open("data/nifty500_secter_list.json") as f:
    sector_data = json.load(f)

NiftySector.insert_many([
    NiftySector(symbol=data["key"], company_name=data["Company Name"], isin_code=data["ISIN Code"], industry=data["Industry"],series=data["Series"])
    for data in sector_data
])